In [186]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Part A

In [169]:
# Load the train and test datasets into a Pandas DataFrame.
trainData = pd.read_csv("/Users/cwbenton/MATH4140/HW/HW3/house-prices/train.csv")
testData = pd.read_csv("/Users/cwbenton/MATH4140/HW/HW3/house-prices/test.csv")

### Part B

In [170]:
# Explore categorical and continuous input variables appropriately
#trainData.info()

In [171]:
features = trainData.drop(columns=["SalePrice"])
target = trainData["SalePrice"]

# Grab all the categorical features (ordinal data is categorical, but coded as numbers)
categorical_features = trainData.select_dtypes(include=object).columns
categorical_features = categorical_features.union(["Id", "MSSubClass", "OverallQual", "OverallCond"])

# Grab all the continuous features (some categorical data are coded as numbers)
continuous_features = trainData.select_dtypes(exclude=object).columns
continuous_features = continuous_features.drop(["Id", "MSSubClass", "OverallQual", "OverallCond"])

In [172]:
# View the categorical features
print(categorical_features)

# View the continuous features
print(continuous_features)

Index(['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'Id', 'KitchenQual', 'LandContour', 'LandSlope',
       'LotConfig', 'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType',
       'MiscFeature', 'Neighborhood', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SaleType', 'Street', 'Utilities'],
      dtype='object')
Index(['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 

In [173]:
# See if there are NaN values for our variables. We will have to do some imputing.
trainData.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [174]:
'''
We can impute all continuous features with the median. As for categorical imputation, we may need
to try different methods. It may not be wise to do most_frequent for all categorical variables. Let's
take a look at them.
'''
trainData['Alley'].fillna('NA', inplace=True)
trainData['MasVnrType'].fillna('None', inplace=True)
trainData['BsmtQual'].fillna('NA', inplace=True)
trainData['BsmtCond'].fillna('NA', inplace=True)
trainData['BsmtExposure'].fillna('NA', inplace=True)
trainData['BsmtFinType1'].fillna('NA', inplace=True)
trainData['BsmtFinType2'].fillna('NA', inplace=True)
trainData['FireplaceQu'].fillna('NA', inplace=True)
trainData['GarageType'].fillna('NA', inplace=True)
trainData['GarageFinish'].fillna('NA', inplace=True)
trainData['GarageQual'].fillna('NA', inplace=True)
trainData['GarageCond'].fillna('NA', inplace=True)
trainData['PoolQC'].fillna('NA', inplace=True)
trainData['Fence'].fillna('NA', inplace=True)
trainData['MiscFeature'].fillna('NA', inplace=True)

'''
We will imupte the rest of the continuous variables with the median, however, for houses without
a garage, I will replace the garage built year with the year that the house was built.
'''
trainData['GarageYrBlt'].fillna(trainData['YearBuilt'], inplace=True)

In [175]:
# See if any NaN values remain
trainData.isnull().sum()

# The remaining nulls can be imputed with the pipeline created below.

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

### Part C/D: Preprocessing and Feature Engineering

In [176]:
'''
Upon looking at all histograms for the continuous features, there is skew. This should make sense
since some houses are larger than others (and some can be really big). Therefore, for imputation, we
will be using the median for all these continuous features.
Also, due to the outliers, we will be using RobustScaler to scale our continuous features.
'''
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create a pipeline for continuous imputation.
continuous_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Create a pipeline for categorical imputation.
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine these two pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_pipeline, continuous_features),
        ('categorical', categorical_pipeline, categorical_features)
    ])

# Create a pipeline with the preprocessing steps and additional steps if needed
cleaning_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# View the pipeline
cleaning_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBa...
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'Id', 'KitchenQual', 'LandContour', 'LandSlope',
       'LotConfig', 'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType',
       'MiscFeature', 'Neighborhood', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SaleType', 'Street', 'Utilities'],
      dtype='object'))]))])

In [177]:
# Fit our pipeline to the model.
train_data_preprocessed = cleaning_pipeline.fit_transform(trainData)

# Store column names after encoding
categorical_columns_encoded = preprocessor.named_transformers_['categorical']['onehot'].get_feature_names_out(categorical_features)

In [178]:
# Put the preprocessed data to a dataframe. Grab the column names and replace them
train_data_preprocessed = pd.DataFrame(train_data_preprocessed.toarray(), columns=list(continuous_features) + list(categorical_columns_encoded))
train_data_preprocessed

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa
0,-0.210526,-0.254076,0.652174,0.243243,1.193303,0.452790,0.0,-0.559829,-0.269652,-0.453608,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.578947,0.030015,0.065217,-0.486486,0.000000,0.834679,0.0,-0.330769,0.538308,0.343643,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,-0.052632,0.437624,0.608696,0.216216,0.986301,0.143910,0.0,-0.074359,-0.142289,-0.327933,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,-0.473684,0.017663,-1.260870,-0.648649,0.000000,-0.235170,0.0,0.106838,-0.468657,-0.247423,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,0.789474,1.181201,0.586957,0.162162,2.130898,0.381186,0.0,0.021368,0.305473,0.113893,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.368421,-0.385746,0.565217,0.162162,0.000000,-0.538435,0.0,0.812821,-0.076617,-0.263132,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1456,0.842105,0.913167,0.108696,-0.162162,0.724505,0.570727,163.0,0.190598,1.095522,1.936181,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1457,-0.157895,-0.107831,-0.695652,0.324324,0.000000,-0.152334,0.0,0.682906,0.319403,0.198331,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1458,-0.052632,0.058918,-0.500000,0.054054,0.000000,-0.469638,1029.0,-0.816239,0.172139,-0.017673,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


### Part E

In [179]:
# Split the train dataset into training and validation, and identify the target variable.
# Here, the target variable is SalePrice -- we want to see if we can predict if a passenger survived.
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(train_data_preprocessed, trainData["SalePrice"], test_size=0.20)

### Part F

In [180]:
# Implement Linear Regression, find best hyperparameters, and report the appropriate evaluation metrics.
from sklearn.linear_model import LinearRegression

# View parameters for Linear Regression
LinearRegression().get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [181]:
# Identify best hyperparameters through a grid.
model_linreg = LinearRegression()

# Explore possible hyperparameters.
param_grid_linreg = {
    'copy_X': [True, False],
    'fit_intercept': [True, False],
    'positive': [True, False]
}

In [182]:
# Use grid search
from sklearn.model_selection import GridSearchCV

grid_f = GridSearchCV(model_linreg, param_grid_linreg, cv=3)

In [183]:
# Fit the training data to our model.
grid_f.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'positive': [True, False]})

In [184]:
# See the best parameters
grid_f.best_params_

{'copy_X': True, 'fit_intercept': True, 'positive': False}

In [185]:
# Predicted sale prices using our validation set.
y_pred_f = grid_f.predict(X_val)
y_pred_f

array([ 60215.28708528, 139789.47709157, 269603.51391657, 110751.83697532,
       234993.14644527, 278061.30888673, 111262.58081172, 249622.40881548,
       173335.18847543, 189989.31908931, 179938.01107551, 313120.24925017,
       230056.42936811, 249616.04644193, 107760.75534411, 131864.32349476,
       102125.27971718, 174138.60656369, 240726.3271282 ,  80631.41585418,
       164989.00835495,  58431.57581943, 174563.73530567, 188794.59410581,
       207622.81960991, 200925.13806415, 394623.00387739, 245118.53911374,
       144773.8992651 , 217090.36120883,  95086.95531442, 148459.96546405,
       139658.39392029, 186877.36528174, 114659.62436164, 149650.73257492,
       124998.34149805,  83006.95460272, 157449.543367  , 148940.51274905,
       271625.04472687, 553860.62981479, 133042.48467251, 201289.39462324,
       250036.57469131, 130098.39197659, 622680.56263802, 144801.95707021,
       170206.66869399, 118520.8064723 , 176966.92547715, 147619.52143912,
       132708.07363784, 1

In [187]:
from sklearn.metrics import mean_squared_error

# For regression, we show the MSE, not the accuracy.
mse = mean_squared_error(Y_val, y_pred_f)
print("Root Mean Squared Error:", np.sqrt(mse))

# On average, the model is about $800 off of the true sale price, which I would say is very good.

Root Mean Squared Error: 808.6043073054261


### Part F

In [ ]:
# Implement Random Forest Regression, find best hyperparameters, and report the appropriate evaluation metrics.